In [1]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

In [11]:
!nvidia-smi

Wed Jun 22 01:54:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   6041MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### [Numba](https://numba.pydata.org/)
"Numpy for GPU" using Nvidia CUDA library

In [2]:
from numba import cuda, float32
import numpy as np
import math

In [3]:
import os
os.environ["NUMBA_CUDA_USE_NVIDIA_BINDING"] = "1"

#### Setup matrix-multipication dataset

In [4]:
x_h = np.arange(100000000).reshape([10000, 10000])
y_h = np.ones([10000, 10000])
z_h = np.zeros([10000, 10000])

#### Slow CPU matrix multiplication

In [5]:
%%time

np.matmul(x_h, y_h, z_h)

CPU times: user 1min 5s, sys: 1.67 s, total: 1min 7s
Wall time: 17.5 s


array([[4.99950000e+07, 4.99950000e+07, 4.99950000e+07, ...,
        4.99950000e+07, 4.99950000e+07, 4.99950000e+07],
       [1.49995000e+08, 1.49995000e+08, 1.49995000e+08, ...,
        1.49995000e+08, 1.49995000e+08, 1.49995000e+08],
       [2.49995000e+08, 2.49995000e+08, 2.49995000e+08, ...,
        2.49995000e+08, 2.49995000e+08, 2.49995000e+08],
       ...,
       [9.99749995e+11, 9.99749995e+11, 9.99749995e+11, ...,
        9.99749995e+11, 9.99749995e+11, 9.99749995e+11],
       [9.99849995e+11, 9.99849995e+11, 9.99849995e+11, ...,
        9.99849995e+11, 9.99849995e+11, 9.99849995e+11],
       [9.99949995e+11, 9.99949995e+11, 9.99949995e+11, ...,
        9.99949995e+11, 9.99949995e+11, 9.99949995e+11]])

#### Setup GPU-specific constructs

In [6]:
TPB = 16

threadsperblock = (TPB, TPB)
blockspergrid_x = math.ceil(z_h.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(z_h.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

#### Slow GPU matrix multiplication

This implementation will perform poorly because individual matrix elements are reloaded multiple times into device memory (slow).

In [7]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B."""
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [8]:
%%time
x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

matmul[blockspergrid, threadsperblock](x_d, y_d, z_d)

z_d.copy_to_host()

CPU times: user 24.6 s, sys: 16.2 s, total: 40.7 s
Wall time: 41 s


array([[4.99950000e+07, 4.99950000e+07, 4.99950000e+07, ...,
        4.99950000e+07, 4.99950000e+07, 4.99950000e+07],
       [1.49995000e+08, 1.49995000e+08, 1.49995000e+08, ...,
        1.49995000e+08, 1.49995000e+08, 1.49995000e+08],
       [2.49995000e+08, 2.49995000e+08, 2.49995000e+08, ...,
        2.49995000e+08, 2.49995000e+08, 2.49995000e+08],
       ...,
       [9.99749995e+11, 9.99749995e+11, 9.99749995e+11, ...,
        9.99749995e+11, 9.99749995e+11, 9.99749995e+11],
       [9.99849995e+11, 9.99849995e+11, 9.99849995e+11, ...,
        9.99849995e+11, 9.99849995e+11, 9.99849995e+11],
       [9.99949995e+11, 9.99949995e+11, 9.99949995e+11, ...,
        9.99949995e+11, 9.99949995e+11, 9.99949995e+11]])

#### Fast GPU matrix multiplication
While a bit more complex, this implementation will perform much faster because we are loading blocks of matrix elements into device memory and using shared memory across those blocks (fast).

In [9]:
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B using CUDA shared memory.

    Reference: https://stackoverflow.com/a/64198479/13697228 by @RobertCrovella
    """
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = float32(0.)
    for i in range(bpg):
        # Preload data into shared memory
        sA[ty, tx] = 0
        sB[ty, tx] = 0
        if y < A.shape[0] and (tx + i * TPB) < A.shape[1]:
            sA[ty, tx] = A[y, tx + i * TPB]
        if x < B.shape[1] and (ty + i * TPB) < B.shape[0]:
            sB[ty, tx] = B[ty + i * TPB, x]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[ty, j] * sB[j, tx]

        # Wait until all threads finish computing
        cuda.syncthreads()
    if y < C.shape[0] and x < C.shape[1]:
        C[y, x] = tmp

In [10]:
%%time
x_d = cuda.to_device(x_h)
y_d = cuda.to_device(y_h)
z_d = cuda.to_device(z_h)

fast_matmul[blockspergrid, threadsperblock](x_d, y_d, z_d)

z_d.copy_to_host()

CPU times: user 3.41 s, sys: 1.73 s, total: 5.14 s
Wall time: 5.14 s


array([[4.99928960e+07, 4.99928960e+07, 4.99928960e+07, ...,
        4.99928960e+07, 4.99928960e+07, 4.99928960e+07],
       [1.49990816e+08, 1.49990816e+08, 1.49990816e+08, ...,
        1.49990816e+08, 1.49990816e+08, 1.49990816e+08],
       [2.49990416e+08, 2.49990416e+08, 2.49990416e+08, ...,
        2.49990416e+08, 2.49990416e+08, 2.49990416e+08],
       ...,
       [9.99893041e+11, 9.99893041e+11, 9.99893041e+11, ...,
        9.99893041e+11, 9.99893041e+11, 9.99893041e+11],
       [9.99907918e+11, 9.99907918e+11, 9.99907918e+11, ...,
        9.99907918e+11, 9.99907918e+11, 9.99907918e+11],
       [1.00003375e+12, 1.00003375e+12, 1.00003375e+12, ...,
        1.00003375e+12, 1.00003375e+12, 1.00003375e+12]])